In [1]:
import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

H:\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [31]:
def load_model_and_vocabulary(model_path, vocab_path):
    model = tf.keras.models.load_model(model_path)
    with open(vocab_path, "rb") as file:
        vocabulary = pickle.load(file)
    return model, vocabulary


def preprocess_text(text: str) -> str:
    text = text.replace("\n", " ")
    text = " ".join(text.strip().split())
    return text

def generate_sentence(model, vocabulary, input_sentence, min_words=10, max_word=100, temperature=1):
    reverse_vocabulary = {v: k for k, v in vocabulary.items()}
    input_sentence = preprocess_text(input_sentence)

    word_count = len(input_sentence.split())
    
    while word_count <= min_words:
        tokenized_sentence = [vocabulary.get(token, vocabulary['<OOV>']) for token in input_sentence.split()]
        padded_sentence = pad_sequences([tokenized_sentence], maxlen=n_max, padding='pre', value=vocabulary['<PAD>'])

        predictions = model.predict(padded_sentence, verbose=3)[0]
        predictions[0] = 0
        predictions[1] = 0
        predictions = np.array([np.power(p, 1 / temperature) for p in predictions])
        predictions = predictions / np.sum(predictions)
    

        next_word_idx = np.random.choice(range(len(predictions)), p=predictions)
        next_word = reverse_vocabulary.get(next_word_idx, '<OOV>')

        input_sentence += ' ' + next_word
        
        word_count = len(input_sentence.split())
        if word_count > min_words and input_sentence[-1] not in [".", "!", "?"]:
            min_words += 1
            
        if word_count >= max_word:
            return input_sentence

    return input_sentence

In [29]:
model_path = "../LSTM_LM3/"
vocab_path = "../vocab.pickle"
n_max = 10
nb_to_generate = 10

model, vocabulary = load_model_and_vocabulary(model_path, vocab_path)

In [34]:
input_sentence = "J'adore l'eau"
for i in range(nb_to_generate):
    completed_sentence = generate_sentence(model, vocabulary, input_sentence, temperature=0.7)
    logging.info(completed_sentence)

INFO:root:J'adore l'eau sur le continent africain, elle est en train de répondre aux défis des autres crises suscitées par les préfets.
INFO:root:J'adore l'eau soit généralisée en France. desquelles je voulais à cette situation, en effet, je veux un problème, c'est de la clarté et la coopération décentralisée, équilibrée, inventive, dépoussiérée, dépoussiérée des racismes et de leur compagnonnage le Taoiseach irlandais et la France qui soutient, des CNR thématiques, comme à la fois de cette compétitivité en février 2020, mais à la fois mon amitié : notre langue.
INFO:root:J'adore l'eau sur le terrain des retraites de France qui ont été en commun de manière dans le Pacifique, je dirais, surtout à la fois le logement et en donnant un pays.
INFO:root:J'adore l'eau ou cette émotion de vie ou de désordre. Joseph Ki-Zerbo à ce titre, que je souhaite que nous avons vocation, à 15% pour inscrire nos intérêts et qui permettent aussi de prendre des droits de provenance sur les prochains mois, qu